In [37]:
import pandas as pd
import numpy as np
import string
import re

In [38]:
df = pd.read_csv('../Data/car_price_prediction.csv')
df.head(25)

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
5,45802912,39493,891,HYUNDAI,Santa FE,2016,Jeep,Yes,Diesel,2,160931 km,4.0,Automatic,Front,04-May,Left wheel,White,4
6,45656768,1803,761,TOYOTA,Prius,2010,Hatchback,Yes,Hybrid,1.8,258909 km,4.0,Automatic,Front,04-May,Left wheel,White,12
7,45816158,549,751,HYUNDAI,Sonata,2013,Sedan,Yes,Petrol,2.4,216118 km,4.0,Automatic,Front,04-May,Left wheel,Grey,12
8,45641395,1098,394,TOYOTA,Camry,2014,Sedan,Yes,Hybrid,2.5,398069 km,4.0,Automatic,Front,04-May,Left wheel,Black,12
9,45756839,26657,-,LEXUS,RX 350,2007,Jeep,Yes,Petrol,3.5,128500 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12


<h1>Data Cleaning Rules</h1>

1. Remove columns: ID, Levy, Model
2. Convert 'engine volume' mixed type column into: engine volume column and HasTurbo column
3. Change mileage into type int, remove the unit 'km' (just change the column title)
4. Remove all rows that has any null data: X rows removed (Y% of all rows)

In [36]:
#1 Remove the 3 columns

df_rem = df.copy()
del df_rem['ID']
del df_rem['Levy']
del df_rem['Model']
df_rem.head()

,Price,Manufacturer,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,13328,LEXUS,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,16621,CHEVROLET,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,8467,HONDA,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,3607,FORD,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,11726,HONDA,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [76]:
#2 Convert 'Engine volume' to 'Engine Volume' and 'HasTurbo'

full_eng = df_rem['Engine volume']
split_eng = []
for i in range(len(full_eng)):
    split_eng.append(list(np.zeros(2).astype(int)))

for i in range(len(split_eng)):
    for j in range(len(full_eng[i].split())):
        split_eng[i][j] = full_eng[i].split()[j]

In [87]:
new_columns_df = pd.DataFrame(split_eng, columns=['Engine Volume', 'HasTurbo'])

engine_volume_pos = df_rem.columns.get_loc('Engine volume')

df_split_engine = pd.concat([df_rem.iloc[:, :engine_volume_pos], new_columns_df, df_rem.iloc[:, engine_volume_pos+1:]], axis=1)
df_split_engine['Engine Volume'] = df_split_engine['Engine Volume'].astype(float)

In [94]:
has_turbo = df_split_engine['HasTurbo']
turbo_bool = [has_turbo[i]=='Turbo' for i in range(len(has_turbo))]
df_split_engine['HasTurbo'] = turbo_bool
df_split_engine.head(25)

,Price,Manufacturer,Prod. year,Category,Leather interior,Fuel type,Engine Volume,HasTurbo,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,13328,LEXUS,2010,Jeep,Yes,Hybrid,3.5,False,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,16621,CHEVROLET,2011,Jeep,No,Petrol,3.0,False,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,8467,HONDA,2006,Hatchback,No,Petrol,1.3,False,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,3607,FORD,2011,Jeep,Yes,Hybrid,2.5,False,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,11726,HONDA,2014,Hatchback,Yes,Petrol,1.3,False,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
5,39493,HYUNDAI,2016,Jeep,Yes,Diesel,2.0,False,160931 km,4.0,Automatic,Front,04-May,Left wheel,White,4
6,1803,TOYOTA,2010,Hatchback,Yes,Hybrid,1.8,False,258909 km,4.0,Automatic,Front,04-May,Left wheel,White,12
7,549,HYUNDAI,2013,Sedan,Yes,Petrol,2.4,False,216118 km,4.0,Automatic,Front,04-May,Left wheel,Grey,12
8,1098,TOYOTA,2014,Sedan,Yes,Hybrid,2.5,False,398069 km,4.0,Automatic,Front,04-May,Left wheel,Black,12
9,26657,LEXUS,2007,Jeep,Yes,Petrol,3.5,False,128500 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12


In [117]:
#3 Change column type of 'Mileage' to int and remove km

# First, check that the only units used is km, and not other units such as miles
split_mileage = df['Mileage'].str.split(' ', expand=True)
    
k = (split_mileage[1] != 'km').sum()
k

0

In [122]:
# We know for sure that all units are in km. Time to remove it.
df_int_mi = df_split_engine.copy()
df_int_mi['Mileage'] = df_int_mi['Mileage'].str.replace(' km', '', regex=False).astype(int)
df_int_mi

,Price,Manufacturer,Prod. year,Category,Leather interior,Fuel type,Engine Volume,HasTurbo,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,13328,LEXUS,2010,Jeep,Yes,Hybrid,3.5,False,186005,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,16621,CHEVROLET,2011,Jeep,No,Petrol,3.0,False,192000,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,8467,HONDA,2006,Hatchback,No,Petrol,1.3,False,200000,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,3607,FORD,2011,Jeep,Yes,Hybrid,2.5,False,168966,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,11726,HONDA,2014,Hatchback,Yes,Petrol,1.3,False,91901,4.0,Automatic,Front,04-May,Left wheel,Silver,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,8467,MERCEDES-BENZ,1999,Coupe,Yes,CNG,2.0,True,300000,4.0,Manual,Rear,02-Mar,Left wheel,Silver,5
19233,15681,HYUNDAI,2011,Sedan,Yes,Petrol,2.4,False,161600,4.0,Tiptronic,Front,04-May,Left wheel,Red,8
19234,26108,HYUNDAI,2010,Jeep,Yes,Diesel,2.0,False,116365,4.0,Automatic,Front,04-May,Left wheel,Grey,4
19235,5331,CHEVROLET,2007,Jeep,Yes,Diesel,2.0,False,51258,4.0,Automatic,Front,04-May,Left wheel,Black,4


In [127]:
#4 Remove null data, it seems we started with a full set!
df_cleaned = df_int_mi.dropna()
df_cleaned

,Price,Manufacturer,Prod. year,Category,Leather interior,Fuel type,Engine Volume,HasTurbo,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,13328,LEXUS,2010,Jeep,Yes,Hybrid,3.5,False,186005,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,16621,CHEVROLET,2011,Jeep,No,Petrol,3.0,False,192000,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,8467,HONDA,2006,Hatchback,No,Petrol,1.3,False,200000,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,3607,FORD,2011,Jeep,Yes,Hybrid,2.5,False,168966,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,11726,HONDA,2014,Hatchback,Yes,Petrol,1.3,False,91901,4.0,Automatic,Front,04-May,Left wheel,Silver,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,8467,MERCEDES-BENZ,1999,Coupe,Yes,CNG,2.0,True,300000,4.0,Manual,Rear,02-Mar,Left wheel,Silver,5
19233,15681,HYUNDAI,2011,Sedan,Yes,Petrol,2.4,False,161600,4.0,Tiptronic,Front,04-May,Left wheel,Red,8
19234,26108,HYUNDAI,2010,Jeep,Yes,Diesel,2.0,False,116365,4.0,Automatic,Front,04-May,Left wheel,Grey,4
19235,5331,CHEVROLET,2007,Jeep,Yes,Diesel,2.0,False,51258,4.0,Automatic,Front,04-May,Left wheel,Black,4


In [128]:
# Save df_cleaned to a new file
df_cleaned.to_csv('cleaned_car_price_prediction.csv', index=False)